# Step 3 - Model hosting using a custom container

In this step, we will host your model behind a SageMaker endpoint.  Depending on the format of the model, you could build your own container to host the model or host the model using a built-in container such as the SageMaker TensorFlow container.  First, let's create a working folder called **MY_PROJECT** and copy over some boilerplate files.

In [ ]:
%%sh

DIRECTORY=MY_PROJECT

target_dir=$DIRECTORY
if [ ! -d "$target_dir" ]; then
    mkdir $target_dir
fi

target_dir=$DIRECTORY/test_data
if [ ! -d "$target_dir" ]; then
    mkdir $target_dir
fi


cp -r ./container ./$DIRECTORY/
cp run_docker.ipynb ./$DIRECTORY/run_docker.ipynb



We can build a separate container for serving the model, or create a single container for both serving and hosting.  For the purpose of demonstrating how the serving process is different from the training, we will create a separate container here.

Open up the container folder inside the MY_PROJECT folder, similar to the training container folder and file structure, you will see the following folders and files
 - **code_base**:  this folder contains the files needed to run a flask server and inference code. The content in this folder will be copy to the working folder when the Dockfile is built.  You will see the following files in the **code_base** directory

     - **nginx.conf** contains configuration details for running nginx server. There is no need to change this file
     - **wsgi.py** instantiates the flask server which is implemented by the predictor.py file
     - **predictor.py** is the file that implements a flask server to do inferences. It's the file that you will modify to implement the scoring for your own algorithm. Specifically, you will need to make the following changes:
        - import any packages as needed.  e.g. from sklearn.externals import joblib
        - enter a file name for the **model_name** variable. This is the model file to be loaded.
        - modify the **ScoringService->get_model()** function to load the model
        - modify the **ScoringService->transformation()** to process the input data as needed. You might not need to modify this function if the existing code already meets your needs.
     - **serve** implements the scoring service shell. You don't necessarily need to modify it for various algorithms. It starts nginx and gunicorn with the correct configurations and then simply waits until gunicorn exits
 - **Dockerfile**:  This is the Dockerfile used to build docker image.  Make changes as needed such as installing additional packages.
 - **build_and_push.sh**:  This shell script builds the docker image and push to your AWS ECR. You will need to change the image name to something unique 
 

## 1.  Build custom container

Now you are ready to build the container. Open the **build_and_push.sh** file and change the name of the image.  Run **build_and_push.sh** in a terminal by typing **sh build_and_push.sh** to build a container and push to ECR.  Make sure you are in the right working directory below.

`/home/ec2-user/SageMaker/SageMaker-Migration-Workshop/step-3/MY_PROJECT/container`

You can list the docker images after it is built by typing the command below

`docker images`


## 2. Local docker image testing

First, we will download saved model from S3 bucket to local folder structure created for testing docker image locally.

Enter a name for the modelpath for the model artifact in S3 before running the cell.  You can find the path of the model in the **SageMaker console->training jobs**. Select the training job and you can find the path to the model artifact under the **Output** section

In [ ]:
%%sh

# copy model from S3 to local directory for testing
modelname=model.tar.gz
modelpath=s3://sagemaker-demo-dyping/timeseries-rf/models/timeseries-rf-2019-01-11-03-26-06-475/output/model.tar.gz
    
aws s3 cp $modelpath $(pwd)/MY_PROJECT/container/opt/ml/model/$modelname

cd $(pwd)/MY_PROJECT/container/opt/ml/model/
tar -xvf $modelname
rm $modelname


#### 2.1 Run docker image locally for testing.

- Run the following command in a terminal window. Make sure you provide a name for the serving docker image.  This starts the docker container locally on your notebook instance.


`cd /home/ec2-user/SageMaker/SageMaker-Migration-Workshop/step-3`

`image=<image name>`
  
`docker run -v $(pwd)/MY_PROJECT/container/opt/ml:/opt/ml -p 8080:8080 --rm ${image} serve`



#### 2.2 Invoke locally running Docker container
After the docker image is running in a separate notebook, let's call the docker container.  

1. copy the test file to the test_data directory
2. Change the payload to the name of the test file and specify the content type (e.g. text/csv). 

You can status in the terminal window where the docker container is running

In [ ]:
%%sh

cd MY_PROJECT/test_data/

payload=test.csv
content=text/csv

curl --data-binary @${payload} -H "Content-Type: ${content}" -v http://localhost:8080/invocations

## 3. Create model and endpoint in SageMaker

Now the image has been created and locally tested, we will now create a model in SageMaker and host it behind an API endpoint

In [ ]:
import sagemaker
import boto3
from time import gmtime, strftime
from sagemaker import get_execution_role

sagemaker_session=sagemaker.Session()
role = get_execution_role()

#### 3.1 Get the container image url

First we need to create a name for the model.  Make sure the name is unique.  

In [ ]:

image_tag = "timeseries-rf-serve"
account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
region = sagemaker_session.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, image_tag)
model_name = "timeseries-model-1"
print(image)

#### 3.2 Create Model

To create a model in SageMaker, we need 2 pieces of information

1. image url for the serving container
2. Path to the model artifact in S3


In [ ]:
client = boto3.client('sagemaker')

# name of the training job that create the model artifacts
job_name ="<name of training job>"
job_name="timeseries-rf-2019-01-11-03-26-06-475"

#you can use the training job name to look up the S3 location for the saved model artifact. 
#You can also find that information in the management console

info = client.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

primary_container = {
    'Image': image,
    'ModelDataUrl': model_data
}

#Create the model
create_model_response = client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn']) 

#### 3.2 Create Endpoint Configuration

Next we need to create an endpoint configuration for hosting the model

In [ ]:
endpoint_config_name = "<name of endpoint config>"
endpoint_config_name = "timeseries-rf-config"
print(endpoint_config_name)


create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

#### 3.3 Create Endpoint

Finally, we can now create the endpoint.

In [ ]:
%%time
import time

endpoint_name = "<name of the endpoint>"
endpoint_name = "timeseries-rf-api-1"

print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

In [ ]:
print(endpoint_name)

### 3.4 Test endpoint

First load the test data as payload. Modify the code as needed depending on whether 

In [ ]:
import boto3
import pandas as pd


file_name = 'MY_PROJECT/test_data/test.csv' #customize to your test file

#input_df = pd.read_csv(file_name, index_col=None, header=None)

input_df = pd.read_csv(file_name)

input_df=input_df.set_index('timestamp_loc')

#payload = input_df.to_csv(index=None, header=None)
payload = input_df.to_csv()

print(payload)

Next invoke the endpoint with payload

In [ ]:
runtime_client = boto3.client('runtime.sagemaker')

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read()
result = result.decode("utf-8")
result = result.split(',')
#label = payload.strip(' ').split()[0]
#print ('Input: ',label,'\nPrediction: ', result[0])
print(result[0])

### 3.5 Batch Transform

In [ ]:
import sagemaker

model_name = 'credit-model-2018-12-20-17-46-53-438-model'
input_location = 's3://sagemaker-demo-dyping/sk-lab/batch/test1.csv'
output_location = 's3://sagemaker-demo-dyping/sk-lab/batch/output/'

transformer =sagemaker.transformer.Transformer(
    base_transform_job_name='Batch-Transform',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    output_path=output_location
    )
# To start a transform job:
transformer.transform(input_location, content_type='text/csv', split_type='Line')
# Then wait until transform job is completed
transformer.wait()